In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 50)

In [ ]:
from utility import *
from trends import *
from plot import *

1. The features in the dataset are continuous. They are as follows:
    - Power consumed by different components
    - Factors influencing power consumption 
    - Time series in an interval of 5 minutes for 2 vessels, spanning across a year. That makes it $12*24*365 = 105120$ data points for each vessel.


In [ ]:
# Read the data
df = pd.read_csv('data/data.csv', header = 0)
df.head()

In [ ]:
# Check the data types and column names
df.dtypes

# Vessel - Level Analysis

In [ ]:
dfv = pick_vessel(df, 'Vessel 1')

In [ ]:
missing_values = dfv.isna().sum()
missing_values.plot(kind='bar',figsize=(12,5), title='Missing Values')  # Plot the missing values

### Impute the missing Values:
1. For features that have $<1%$  missing values, impute by interpolating them using closest non-missing values as the features are all usually smooth within the 5mins time intervals in which they are recorded
2. For features that have $>20%$ missing values, impute by using median of the column/feautre

In [ ]:
# Imputing the missing values
missing_values

In [ ]:
col_to_interpolate = dfv.columns.difference(['Depth (m)', 'Start Time', 'End Time', 'Vessel Name']) # Columns to interpolate

In [ ]:
impute_time_series(dfv, col_to_interpolate) # Imputation via interpolation for the columns with < 1% missing values as they are likely to be continuous in time
dfv.isna().sum()

In [ ]:
median_depth = dfv['Depth (m)'].median() # impution via median value as more than 20% of the values are missing
dfv['Depth (m)'].fillna(median_depth, inplace=True)
dfv.isna().sum()

In [ ]:
dfv.head()

### Multi-Collinearity Check

In [ ]:
# Correlation matrix
dfv_sub = dfv.iloc[:,3:]
corr = dfv_sub.corr()

In [ ]:
high_corr = []
for i in corr.columns:
    high_corr.append(corr[(corr[i] > 0.8) | (corr[i] < -0.8)][i])
print(f'The highly correlated columns are:{high_corr}')

In [ ]:
# Plot the correlation matrix
fig, ax = plt.subplots(figsize=(24,20))
sns.heatmap(data = corr[(corr > 0.8) | (corr < -0.8)], vmin=-1,vmax=1, cmap='coolwarm', ax = ax, annot= True)
ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
ax.set_title('Correlation Matrix')
plt.show()

**Comments (Vessel 1):**
1. Diesel Generators' Power are highly correlated to corresponding Main Engine's Fuel flow rate 
2. Propulsion power is a linear combination of Port Side Propulsion Power and Starboard Side Propulsion Power 
3. Speed through water and Speed over ground are positively correlated to the Propulsion Power
4. Sea water temperature reducing as the latitude is increasing completely makes sense. This is because, temperatures are lower as one moves towards the poles

**Comments (Vessel 2):**
1. Diesel

### Trend and seasonality analysis

In [ ]:
# Resampling the data for hourly, daily, weekly and seasonal trends
hourly_df, daily_df, weekly_df, monthly_df = resample(dfv)

| Column Name |
|-------------|
| Power Galley 1 (MW) |
| Power Galley 2 (MW) |
| Power Service (MW) |
| HVAC Chiller 1 Power (MW) |
| HVAC Chiller 2 Power (MW) |
| HVAC Chiller 3 Power (MW) |
| Scrubber Power (MW) |
| Sea Temperature (Celsius) |
| Boiler 1 Fuel Flow Rate (L/h) |
| Boiler 2 Fuel Flow Rate (L/h) |
| Incinerator 1 Fuel Flow Rate (L/h) |
| Diesel Generator 1 Power (MW) |
| Diesel Generator 2 Power (MW) |
| Diesel Generator 3 Power (MW) |
| Diesel Generator 4 Power (MW) |
| Latitude (Degrees) |
| Longitude (Degrees) |
| Relative Wind Angle (Degrees) |
| True Wind Angle (Degrees) |
| Depth (m) |
| Relative Wind Direction (Degrees) |
| True Wind Direction (Degrees) |
| Draft (m) |
| Speed Over Ground (knots) |
| True Wind Speed (knots) |
| Relative Wind Speed (knots) |
| Speed Through Water (knots) |
| Local Time (h) |
| Trim (m) |
| Propulsion Power (MW) |
| Port Side Propulsion Power (MW) |
| Starboard Side Propulsion Power (MW) |
| Bow Thruster 1 Power (MW) |
| Bow Thruster 2 Power (MW) |
| Bow Thruster 3 Power (MW) |
| Stern Thruster 1 Power (MW) |
| Stern Thruster 2 Power (MW) |
| Main Engine 1 Fuel Flow Rate (kg/h) |
| Main Engine 2 Fuel Flow Rate (kg/h) |
| Main Engine 3 Fuel Flow Rate (kg/h) |
| Main Engine 4 Fuel Flow Rate (kg/h) |

In [ ]:
# Pick a feature from the above list to visualize the trend over
trend_plot(hourly_df, daily_df, weekly_df, monthly_df, 'Propulsion Power (MW)')

In [ ]:
# Pick 2 features from the above list to visualize their relationship with each other over time
pair_plot(daily_df, hourly_df, weekly_df, monthly_df, 'Propulsion Power (MW)', 'Relative Wind Speed (knots)')

In [ ]:
weekly_df.reset_index(inplace=True)
weekly_df.head()

In [ ]:
# Generate column Table
table = '| Column Name |\n|-------------|\n'
for column in weekly_df.columns:
    table += f'| {column} |\n'
print(table)